<a href="https://colab.research.google.com/github/AhmedSSoliman/Leveraging-Pretrained-Language-Models-for-Code-Generation/blob/master/ELECTRA_Marian_DJANGO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import torch
torch.cuda.empty_cache()

# **Install dependencies**

In [ ]:
%%capture
!pip install tensorflow
!pip install keras
!pip install netron
!pip install nltk

!pip install git+https://github.com/huggingface/transformers
!pip install datasets
!pip install nlp
!pip install rouge-score
#!pip install sacrebleu
!pip install git-python
!pip install sentencepiece
#!pip install torchdata
#!pip install transformers
!pip install sacrebleu==1.5.1
!pip install tree-sitter
!pip install evaluate

# **Import Libraries**

In [ ]:
import transformers
import datasets
from datasets import load_dataset, load_metric
import logging
from transformers import BertTokenizer, GPT2Tokenizer, GPT2TokenizerFast, EncoderDecoderModel, Trainer, TrainingArguments, BertTokenizerFast
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

logging.basicConfig(level=logging.INFO)

import types
import argparse
import logging
from functools import partial
import json

import torch
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    BertGenerationConfig,
    BertGenerationEncoder,
    BertTokenizer,
    EncoderDecoderModel,
    EncoderDecoderConfig,
    GPT2LMHeadModel,
    GPT2TokenizerFast,
    Trainer,
    TrainingArguments,
    get_cosine_schedule_with_warmup,
)

import sacrebleu
import random
import numpy as np
#from tensorboardX import SummaryWriter


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
%cd /content/drive/MyDrive/models/CodeGeneration/New/DJANGO/ELECTRA-Marian

/content/drive/MyDrive/models/CodeGeneration/New/DJANGO/ELECTRA-Marian


# **Load Data**

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("AhmedSSoliman/DJANGO")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/AhmedSSoliman___csv/AhmedSSoliman--DJANGO-3a814141e88829f9/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
train_dataset= raw_dataset["train"]
val_dataset = raw_dataset["validation"]
test_dataset = raw_dataset["test"]

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'nl', 'code'],
        num_rows: 16000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'nl', 'code'],
        num_rows: 1805
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'nl', 'code'],
        num_rows: 1000
    })
})

# **Tokenizer**

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/electra-small-discriminator", use_fast = True)
tokenizer

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='google/electra-small-discriminator', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

# **Model**

In [ ]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained("google/electra-small-discriminator", "Helsinki-NLP/opus-mt-en-nl")
model

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Some weights of the model checkpoint at Helsinki-NLP/opus-mt-en-nl were not used when initializing MarianForCausalLM: ['model.encoder.layers.5.fc2.bias', 'model.encoder.layers.0.self_attn.q_proj.bias', 'model.encoder.layers.2.fc2.bias', 'model.encoder.layers.0.self_attn_layer_norm.bias', 'model.encoder.layers.5.self_attn.v_proj.weight', 'model.shared.weight', 'model.encoder.layers.1.self_attn.v_proj.weight', 'model.encoder.layers.0.self_attn.k_proj.weight', 'model.encoder.layers.3.self_attn.q_proj.weight', 'model.encoder.layers.1.self_attn.q_proj.weight', 'model.encoder.layers.3.fc2.weight', 'model.encoder.layers.2.self_attn.q_proj.bias', 'model.encoder.layers.0.self_attn.v_proj.weight', 'model.encoder.layers.4.self_attn.k_proj.weight', 'model.encoder.layers.4.self_attn.v_proj.bias', 'model.encoder.layers.4.fc2.weight', 'model.encoder.layers.4.final_layer_norm.weight', 'model.encoder.layers.5.self_attn.out_proj.weight', 'model.encoder.layers.4.fc1.bias', 'model.encoder.layers.5.fc1.wei

EncoderDecoderModel(
  (encoder): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=256,

In [ ]:
model.config

EncoderDecoderConfig {
  "_commit_hash": null,
  "decoder": {
    "_name_or_path": "Helsinki-NLP/opus-mt-en-nl",
    "_num_labels": 3,
    "activation_dropout": 0.0,
    "activation_function": "swish",
    "add_bias_logits": false,
    "add_cross_attention": true,
    "add_final_layer_norm": false,
    "architectures": [
      "MarianMTModel"
    ],
    "attention_dropout": 0.0,
    "bad_words_ids": [
      [
        67027
      ]
    ],
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classif_dropout": 0.0,
    "classifier_dropout": 0.0,
    "cross_attention_hidden_size": null,
    "d_model": 512,
    "decoder_attention_heads": 8,
    "decoder_ffn_dim": 2048,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 6,
    "decoder_start_token_id": 67027,
    "decoder_vocab_size": 67028,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "dropout": 0.1,
    "early_stopping": false,
    "encoder_attention_heads": 8,
    "encoder_ffn_dim": 2048,
    "encoder_layerdrop": 0.

In [ ]:
model.base_model.config.encoder.num_hidden_layers = 6

In [ ]:
model.tie_weights = True

In [ ]:
model.base_model.config.decoder_start_token_id = model.base_model.config.decoder.bos_token_id

In [ ]:
model.base_model.config.bos_token_id = tokenizer.bos_token_id
model.base_model.config.eos_token_id = tokenizer.eos_token_id
model.base_model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
model.config.max_length =512

In [ ]:
model.save_pretrained("ELECTRA-Marian-base-model")

In [ ]:
from transformers import AutoModelForSeq2SeqLM
#model = AutoModelForSeq2SeqLM.from_pretrained("./ELECTRA-Marian-base-model")
model=AutoModelForSeq2SeqLM.from_pretrained("./ELECTRA-marian-training")
model

EncoderDecoderModel(
  (encoder): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=256,

In [ ]:
model.base_model.config.decoder_start_token_id

0

In [ ]:
model.base_model.config.pad_token_id

0

In [ ]:
tokenizer.pad_token_id

0

In [ ]:
model.config

EncoderDecoderConfig {
  "_commit_hash": null,
  "_name_or_path": "./ELECTRA-Marian-base-model",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "Helsinki-NLP/opus-mt-en-nl",
    "_num_labels": 3,
    "activation_dropout": 0.0,
    "activation_function": "swish",
    "add_bias_logits": false,
    "add_cross_attention": true,
    "add_final_layer_norm": false,
    "architectures": [
      "MarianMTModel"
    ],
    "attention_dropout": 0.0,
    "bad_words_ids": [
      [
        67027
      ]
    ],
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classif_dropout": 0.0,
    "classifier_dropout": 0.0,
    "cross_attention_hidden_size": null,
    "d_model": 512,
    "decoder_attention_heads": 8,
    "decoder_ffn_dim": 2048,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 6,
    "decoder_start_token_id": 67027,
    "decoder_vocab_size": 67028,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "dropout": 0.1,
    "early_stoppi

# **Evaluator**

In [ ]:
%cd Utils

/content/drive/MyDrive/models/CodeGeneration/New/DJANGO/ELECTRA-Marian/Utils


In [ ]:
from google.colab import files
import evaluator
evaluator = evaluator.CodeGenerationEvaluator(tokenizer, device,smooth_bleu=True)

/content/drive/MyDrive/models/CodeGeneration/New/DJANGO/ELECTRA-Marian/Utils/evaluator.py:32: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.sacre_bleu: Metric = load_metric('sacrebleu')


In [ ]:
%cd ..

/content/drive/MyDrive/models/CodeGeneration/New/DJANGO/ELECTRA-Marian




# **Prepare the data.**

In [ ]:
encoder_length = 32
decoder_length = 32
batch_size = 1


# map data correctly
def map_to_encoder_decoder_inputs(batch):
    inputs = tokenizer(batch["nl"], padding="max_length", truncation=True, max_length=encoder_length)
    outputs = tokenizer(batch["code"], padding="max_length", truncation=True, max_length=decoder_length)
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    batch["decoder_attention_mask"] = outputs.attention_mask

    """
    # complicated list comprehension here because pad_token_id alone is not good enough to know whether label should be excluded or not
    batch["labels"] = [
        [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
    ]
    """
    assert all([len(x) == encoder_length for x in inputs.input_ids])
    assert all([len(x) == decoder_length for x in outputs.input_ids])

    return batch

In [ ]:
# make train dataset ready
train_data = train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=8, remove_columns=['nl', 'code'],
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)



  0%|          | 0/2000 [00:00<?, ?ba/s]

In [ ]:
# same for validation dataset
val_data = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=4, remove_columns=['nl', 'code'],
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


  0%|          | 0/250 [00:00<?, ?ba/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       max_length=512,padding=True, ####new
                                       model = model)

# **Training**

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./ELECTRA-marian-training1",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    num_train_epochs=10,
    do_train=True,
    do_eval=True,
    fp16=True,
    overwrite_output_dir=True,
    learning_rate = 1e-5,
    weight_decay=0.01,
    warmup_ratio = 0.05,
    seed = 1995,
    save_total_limit = 2,
    load_best_model_at_end = True,
    #push_to_hub=True,
)
#

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=evaluator,
    data_collator = data_collator,
    train_dataset=train_data,
    eval_dataset =val_data,

)
#    save_total_limit=3,
#

Using cuda_amp half precision backend


In [ ]:
# ELECTRA-Marian-Training-1
trainer.train()

The following columns in the training set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 16000
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 20000
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `p

Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,0.276800,0.326564,57.129997,56.243272,69.550147,60.867085,52.997963,67.631827,84.795241,67.982106,59.367946,52.628582
2,0.254700,0.309880,63.708308,63.056128,76.105616,67.310603,59.514289,68.872176,85.556110,74.869474,66.050324,59.367115
3,0.229600,0.297664,64.798095,63.967569,76.756713,68.195148,60.965638,69.300396,85.788301,75.532377,67.025226,60.430446
4,0.200100,0.289870,71.960637,71.360042,83.481429,75.063390,68.158292,71.300249,87.393370,82.603720,74.106484,67.830688


The following columns in the evaluation set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
Saving model checkpoint to ./ELECTRA-marian-training1/checkpoint-2000
Configuration saved in ./ELECTRA-marian-training1/checkpoint-2000/config.json
Model weights saved in ./ELECTRA-marian-training1/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in ./ELECTRA-marian-training1/checkpoint-2000/tokenizer_config.json
Special tokens file saved in ./ELECTRA-marian-training1/checkpoint-2000/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2319: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignore

Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,0.276800,0.326564,57.129997,56.243272,69.550147,60.867085,52.997963,67.631827,84.795241,67.982106,59.367946,52.628582
2,0.254700,0.309880,63.708308,63.056128,76.105616,67.310603,59.514289,68.872176,85.556110,74.869474,66.050324,59.367115
3,0.229600,0.297664,64.798095,63.967569,76.756713,68.195148,60.965638,69.300396,85.788301,75.532377,67.025226,60.430446
4,0.200100,0.289870,71.960637,71.360042,83.481429,75.063390,68.158292,71.300249,87.393370,82.603720,74.106484,67.830688
5,0.202400,0.288515,66.408262,65.905713,78.252151,69.878749,62.338572,70.570793,86.737653,77.082219,68.748776,62.378168
6,0.187200,0.282070,68.610940,68.179235,80.267302,71.917744,64.646258,70.819503,86.887933,79.222081,70.916015,64.703267
7,0.174400,0.282296,69.710374,69.336497,80.830973,73.011550,65.735857,72.234126,87.997952,79.781371,71.930002,65.944307
8,0.172300,0.275766,70.735417,70.418702,81.844980,73.846592,66.862047,72.086200,87.927455,80.884265,72.946959,67.049852
9,0.166900,0.274685,70.487838,70.225852,81.454891,73.695673,66.590126,72.013763,87.879404,80.490514,72.728200,66.900850
10,0.157500,0.274240,70.323272,70.171477,81.493647,73.602044,66.357284,71.644532,87.609261,80.532939,72.726346,66.821477


Saving model checkpoint to ./ELECTRA-marian-training1/checkpoint-10000
Configuration saved in ./ELECTRA-marian-training1/checkpoint-10000/config.json
Model weights saved in ./ELECTRA-marian-training1/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in ./ELECTRA-marian-training1/checkpoint-10000/tokenizer_config.json
Special tokens file saved in ./ELECTRA-marian-training1/checkpoint-10000/special_tokens_map.json
Deleting older checkpoint [ELECTRA-marian-training1/checkpoint-6000] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2319: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces a be

TrainOutput(global_step=20000, training_loss=0.2068538417816162, metrics={'train_runtime': 7846.7767, 'train_samples_per_second': 20.391, 'train_steps_per_second': 2.549, 'total_flos': 925520363520000.0, 'train_loss': 0.2068538417816162, 'epoch': 10.0})

In [ ]:
# ELECTRA-Marian-Training
trainer.train()

The following columns in the training set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 16000
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 160000
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `

Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,1.376300,1.239425,2.203191,1.906826,8.395680,3.562467,1.413251,9.560291,37.718242,7.532761,2.886806,1.217413
2,0.779000,0.757898,12.312874,11.350797,22.744566,15.094053,9.863434,36.264078,63.170344,21.169270,13.638625,9.155042
3,0.611200,0.562777,16.239822,15.290831,25.796314,18.910498,13.651567,48.616289,71.354147,24.182517,17.498952,13.043478
4,0.487500,0.470647,27.887886,26.594496,39.815365,31.258082,24.390734,56.295989,77.229506,37.935923,29.456403,23.501199
5,0.437300,0.424375,34.909961,33.579115,46.974018,38.492417,31.192498,61.034227,80.372459,44.977182,36.550881,30.344913
6,0.371200,0.390502,43.710626,42.223256,56.020453,47.416373,39.873373,65.040797,83.227432,54.053700,45.367905,38.788563
7,0.309700,0.367391,57.833668,56.799111,71.301542,61.667236,53.467562,65.848466,83.961561,69.766857,60.097034,52.855994
8,0.314500,0.352238,38.903482,37.579054,49.907779,42.153264,35.359183,66.642574,84.131158,47.977369,40.293853,34.514176


The following columns in the evaluation set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 1
Saving model checkpoint to ./ELECTRA-marian-training/checkpoint-16000
Configuration saved in ./ELECTRA-marian-training/checkpoint-16000/config.json
Model weights saved in ./ELECTRA-marian-training/checkpoint-16000/pytorch_model.bin
tokenizer config file saved in ./ELECTRA-marian-training/checkpoint-16000/tokenizer_config.json
Special tokens file saved in ./ELECTRA-marian-training/checkpoint-16000/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2319: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignore

Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,1.376300,1.239425,2.203191,1.906826,8.395680,3.562467,1.413251,9.560291,37.718242,7.532761,2.886806,1.217413
2,0.779000,0.757898,12.312874,11.350797,22.744566,15.094053,9.863434,36.264078,63.170344,21.169270,13.638625,9.155042
3,0.611200,0.562777,16.239822,15.290831,25.796314,18.910498,13.651567,48.616289,71.354147,24.182517,17.498952,13.043478
4,0.487500,0.470647,27.887886,26.594496,39.815365,31.258082,24.390734,56.295989,77.229506,37.935923,29.456403,23.501199
5,0.437300,0.424375,34.909961,33.579115,46.974018,38.492417,31.192498,61.034227,80.372459,44.977182,36.550881,30.344913
6,0.371200,0.390502,43.710626,42.223256,56.020453,47.416373,39.873373,65.040797,83.227432,54.053700,45.367905,38.788563
7,0.309700,0.367391,57.833668,56.799111,71.301542,61.667236,53.467562,65.848466,83.961561,69.766857,60.097034,52.855994
8,0.314500,0.352238,38.903482,37.579054,49.907779,42.153264,35.359183,66.642574,84.131158,47.977369,40.293853,34.514176
9,0.261300,0.346803,48.068095,46.746668,59.706362,51.602094,44.151368,67.939793,85.036337,57.803587,49.730647,43.377615
10,0.271800,0.343194,53.955464,52.821465,66.020533,57.537319,49.861913,68.263831,85.212553,64.334017,55.801332,49.253197


Saving model checkpoint to ./ELECTRA-marian-training/checkpoint-144000
Configuration saved in ./ELECTRA-marian-training/checkpoint-144000/config.json
Model weights saved in ./ELECTRA-marian-training/checkpoint-144000/pytorch_model.bin
tokenizer config file saved in ./ELECTRA-marian-training/checkpoint-144000/tokenizer_config.json
Special tokens file saved in ./ELECTRA-marian-training/checkpoint-144000/special_tokens_map.json
Deleting older checkpoint [ELECTRA-marian-training/checkpoint-112000] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2319: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces a b

TrainOutput(global_step=160000, training_loss=0.6986664794921875, metrics={'train_runtime': 54923.2659, 'train_samples_per_second': 2.913, 'train_steps_per_second': 2.913, 'total_flos': 925520363520000.0, 'train_loss': 0.6986664794921875, 'epoch': 10.0})

In [ ]:
trainer.save_model()
trainer.save_state()

Saving model checkpoint to ./ELECTRA-marian-training1
Configuration saved in ./ELECTRA-marian-training1/config.json
Model weights saved in ./ELECTRA-marian-training1/pytorch_model.bin
tokenizer config file saved in ./ELECTRA-marian-training1/tokenizer_config.json
Special tokens file saved in ./ELECTRA-marian-training1/special_tokens_map.json


# **Visualize**

In [ ]:
%load_ext tensorboard

### **distilRoBERTa-marian**

In [ ]:
tensorboard --logdir ./distilroberta-Marian-Training/runs

<IPython.core.display.Javascript object>

# **Inference**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
new_model = AutoModelForSeq2SeqLM.from_pretrained("./ELECTRA-marian-training1/")
new_model.to("cuda")

new_tokenizer = AutoTokenizer.from_pretrained("./ELECTRA-marian-training1/", use_fast=True)


In [ ]:
new_model

EncoderDecoderModel(
  (encoder): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=256,

In [ ]:
new_model.config.max_length


512

# **Evaluator**

In [ ]:
%cd Utils

/content/drive/MyDrive/models/CodeGeneration/New/DJANGO/ELECTRA-Marian/Utils


In [ ]:
from google.colab import files
import evaluator
evaluator = evaluator.CodeGenerationEvaluator(new_tokenizer, device,smooth_bleu=True)

/content/drive/MyDrive/models/CodeGeneration/New/DJANGO/ELECTRA-Marian/Utils/evaluator.py:32: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.sacre_bleu: Metric = load_metric('sacrebleu')


In [ ]:
batch_size = 1

# map data correctly
def generate_new_code(batch):
    inputs = new_tokenizer(batch["nl"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = new_model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = new_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_code"] = output_str

    return batch


In [ ]:
results = test_dataset.map(generate_new_code, batched=True, batch_size=batch_size)

#results = test_dataset.map(generate_new_code, batched=True, batch_size=batch_size, remove_columns=["intent"])

  0%|          | 0/1805 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1261: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [ ]:
%cd ..

/content/drive/MyDrive/models/CodeGeneration/New/DJANGO/ELECTRA-Marian


In [ ]:
import re
import string

import datasets
import numpy as np

import evaluate
from evaluate import load

In [ ]:

exact_match = evaluate.load("exact_match")

#refs = ["The cat sat on the mat.", "Theaters are great.", "It's like comparing oranges and apples."]
#preds = ["The cat sat on the mat?", "Theaters are great.", "It's like comparing apples and oranges."]

"""
for i in range(len(results)):
  refs = results["code"][i]
  preds = results["pred_code"][i]
"""

'\nfor i in range(len(results)):\n  refs = results["code"][i]\n  preds = results["pred_code"][i]\n'

In [ ]:
refs = results["code"]
preds = results["pred_code"]

In [ ]:
exact_match_results = exact_match.compute(references=refs, predictions=preds, regexes_to_ignore=["▁", " "], ignore_case=True, ignore_punctuation=True, ignore_numbers=True)
print(round(exact_match_results["exact_match"], 5)*100)

65.319


In [ ]:
with open("./references.txt", 'w') as f:
  for i in range(len(results)):
    nl = results["nl"][i]
    ref = results["code"][i]
    pred = results["pred_code"][i]

    if pred is not None and pred != "":
      if ref is not None and ref != "":
        f.write(ref + '\n')
    else:
      continue


with open("./predictions.txt", 'w') as f:
  for i in range(len(results)):
    nl = results["nl"][i]
    ref = results["code"][i]
    pred = results["pred_code"][i]

    if pred is not None and pred != "":
      if ref is not None and ref != "":
        f.write(pred + '\n')
    else:
      continue


In [ ]:
# Ok Evaluator
bleu_score =0
rouge_score = 0
sacre_bleu = 0

for i in range(len(results)):
  ref = results["code"][i]
  pred = results["pred_code"][i]
  if pred is not None and pred != "":
      if ref is not None and ref != "":
        #bleu_metric = metric.compute(predictions=[pred], references=[[ref]])
        #bleu_score  += bleu_metric["score"]

        bleu_metric = evaluator.evaluate([pred], [ref])
        #bleu_metric = evaluator.evaluateSingle(pred, ref)

        #bleu_score += bleu_metric['BLEU-Unigram-Precision']
        bleu_score += bleu_metric['BLEU']
        rouge_score += bleu_metric['ROUGE-L']
        sacre_bleu += bleu_metric['SacreBLEU']
        #print(bleu_metric['SacreBLEU'])
  else:
    continue

# **Performance Metrics**

**BLEU Score**

In [ ]:
# print("Bleu Score: ",bleu_score/len(results))
print('Bleu Score: {} (scale 0-100)'.format(bleu_score/len(results)))

Bleu Score: 53.01968844718701 (scale 0-100)


**SacreBLEU**

In [ ]:
# print("Bleu Score: ",bleu_score/len(results))
print('Sacre Bleu: {} (scale 0-100)'.format(sacre_bleu/len(results)))

Sacre Bleu: 58.15526577711313 (scale 0-100)


**Rouge Score**

In [ ]:
# print("Bleu Score: ",bleu_score/len(results))
print('ROUGE Score: {} (scale 0-100)'.format(rouge_score/len(results)))

ROUGE Score: 83.90547445301046 (scale 0-100)


**CodeBLEU**

In [ ]:
from google.colab import files

In [ ]:
import calc_code_bleu
ngram_match_score, weighted_ngram_match_score, syntax_match_score, dataflow_match_score, code_bleu_score = calc_code_bleu.calc_code_bleu("references.txt","predictions.txt","python", "keywords")

In [ ]:
print("ngram_match_score: ", ngram_match_score*100)
print("weighted_ngram_match_score: ", weighted_ngram_match_score*100)
print("syntax_match_score: ", syntax_match_score*100)
print("dataflow_match_score: ", dataflow_match_score*100)
print("code_bleu_score: ", code_bleu_score*100)

# **Save the  results**

In [ ]:
#%cd ..

/content/drive/MyDrive/models/CodeGeneration/MarianCG-CoNaLa-Large


In [ ]:
bleu_score = 0
with open("./ELECTRA-marian-training/Output.txt", 'w') as f:
  for i in range(len(results)):
    nl = results["nl"][i]
    ref = results["code"][i]
    pred = results["pred_code"][i]

    if pred is not None and pred != "":
      if ref is not None and ref != "":

        bleu_metric = evaluator.evaluate([pred], [ref])
        f.write( "NL: " + nl + '\n')
        f.write( "ref: " + ref + '\n')
        f.write( "Output: " + pred + '\n')
        f.write("bleu_metric: " + str(bleu_metric["BLEU"]) + '\n')
        f.write(10*"-" + '\n')
        bleu_score  += bleu_metric["BLEU"]

    else:
      continue


# **Test**

In [ ]:
# map data correctly
def generate_code(NL):
    inputs = new_tokenizer([NL], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = new_model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_code = new_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return output_code

In [ ]:

#NL = "decode a hex string to utf-8."
NL = "remove first and last lines of string `s`"
output_code = generate_code(NL)

In [ ]:
output_code

In [ ]:
natural_language = "How I can get rid of None values in dictionary?"
gen_code = generate_code(natural_language)

In [ ]:
gen_code